In [31]:
print("testing python")

testing python


In [32]:
import pandas as pd
import numpy as np

In [56]:
import pandas as pd
# Load raw data (fixed absolute path)
df = pd.read_csv("/Users/shreyarora/Desktop/Developer/ETL_pipeline/student-mental-health-pipeline/data/raw/rawMental.csv")
print(df.head())

        Timestamp Choose your gender   Age What is your course?  \
0  8/7/2020 12:02             Female  18.0          Engineering   
1  8/7/2020 12:04               Male  21.0    Islamic education   
2  8/7/2020 12:05               Male  19.0                  BIT   
3  8/7/2020 12:06             Female  22.0                 Laws   
4  8/7/2020 12:13               Male  23.0         Mathemathics   

  Your current year of Study What is your CGPA? Marital status  \
0                     year 1        3.00 - 3.49             No   
1                     year 2        3.00 - 3.49             No   
2                     Year 1        3.00 - 3.49             No   
3                     year 3        3.00 - 3.49            Yes   
4                     year 4        3.00 - 3.49             No   

  Do you have Depression? Do you have Anxiety? Do you have Panic attack?  \
0                     Yes                   No                       Yes   
1                      No                  Yes  

In [57]:
# Check whether the CSV path used in the notebook exists
import os
path_used = "/Users/shreyarora/Desktop/Developer/mental_health_pipeline/student-mental-health-pipeline/data/raw/rawMental.csv"
path_expected = "/Users/shreyarora/Desktop/Developer/ETL_pipeline/student-mental-health-pipeline/data/raw/rawMental.csv"
print("path_used exists:", os.path.exists(path_used))
print("path_expected exists:", os.path.exists(path_expected))

path_used exists: False
path_expected exists: True


In [35]:
df_clean = df.copy()

print("Working copy created")
print("Rows:", df_clean.shape[0])
print("Columns:", df_clean.shape[1])


Working copy created
Rows: 101
Columns: 11


In [36]:
df_clean.columns = (
    df_clean.columns
    .str.lower()
    .str.strip()
    .str.replace(" ", "_")
)

df_clean.columns


Index(['timestamp', 'choose_your_gender', 'age', 'what_is_your_course?',
       'your_current_year_of_study', 'what_is_your_cgpa?', 'marital_status',
       'do_you_have_depression?', 'do_you_have_anxiety?',
       'do_you_have_panic_attack?',
       'did_you_seek_any_specialist_for_a_treatment?'],
      dtype='object')

In [37]:
df_clean.isnull().sum().sort_values(ascending=False)


age                                             1
timestamp                                       0
choose_your_gender                              0
what_is_your_course?                            0
your_current_year_of_study                      0
what_is_your_cgpa?                              0
marital_status                                  0
do_you_have_depression?                         0
do_you_have_anxiety?                            0
do_you_have_panic_attack?                       0
did_you_seek_any_specialist_for_a_treatment?    0
dtype: int64

In [38]:
critical_columns = ["age", "what_is_your_course?"]

df_clean[critical_columns].isnull().sum()


age                     1
what_is_your_course?    0
dtype: int64

In [39]:
before = df_clean.shape[0]

df_clean = df_clean.dropna(subset=critical_columns)

after = df_clean.shape[0]

print(f"Rows removed: {before - after}")


Rows removed: 1


In [40]:
df_clean["age"] = pd.to_numeric(df_clean["age"], errors="coerce")


In [41]:
df_clean = df_clean[df_clean["age"].between(10, 100)]


In [42]:
binary_map = {
    "Yes": 1, "No": 0,
    "yes": 1, "no": 0,
    "Y": 1, "N": 0,
    True: 1, False: 0
}

binary_columns = [
    "depression",
    "anxiety",
    "panic_attack"
]

for col in binary_columns:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].map(binary_map)


In [43]:
before = df_clean.shape[0]
df_clean = df_clean.drop_duplicates()
after = df_clean.shape[0]

print(f"Duplicates removed: {before - after}")


Duplicates removed: 0


In [44]:
df_clean.info()


<class 'pandas.core.frame.DataFrame'>
Index: 100 entries, 0 to 100
Data columns (total 11 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   timestamp                                     100 non-null    object 
 1   choose_your_gender                            100 non-null    object 
 2   age                                           100 non-null    float64
 3   what_is_your_course?                          100 non-null    object 
 4   your_current_year_of_study                    100 non-null    object 
 5   what_is_your_cgpa?                            100 non-null    object 
 6   marital_status                                100 non-null    object 
 7   do_you_have_depression?                       100 non-null    object 
 8   do_you_have_anxiety?                          100 non-null    object 
 9   do_you_have_panic_attack?                     100 non-null    object 


In [45]:
output_path = "/Users/shreyarora/Desktop/Developer/ETL_pipeline/student-mental-health-pipeline/data/processed/cleaned.csv"
df_clean.to_csv(output_path, index=False)

print("✅ Clean data saved to:", output_path)

✅ Clean data saved to: /Users/shreyarora/Desktop/Developer/ETL_pipeline/student-mental-health-pipeline/data/processed/cleaned.csv


In [46]:
# we will be making a ETL ( extract , transform, load ), we can make this because our data has been cleaned. 

import pandas as pd

def extract():
    """
    Extracts raw data from a CSV file and loads it into memory
    without applying any transformations.

    Parameters:
        source_path (str): Path to the CSV file.

    Returns:
        pd.DataFrame: Raw dataset ready for transformation.
    """
    data = pd.read_csv("/Users/shreyarora/Desktop/Developer/ETL_pipeline/student-mental-health-pipeline/data/processed/cleaned.csv")
    return data

In [53]:
# the second step is to trasnform our data for the pipeline. in this case we will be getting the information for which major has the msot depression. 

def transform(data):
    df = data
    unique_majors = df["what_is_your_course?"].drop_duplicates()
    total_number_of_unique_majors = len(unique_majors)

    # this tells us the diffrent types of courses and their total amount

    # print("the total number of majors is ", total_number_of_unique_majors)
    # print("these are the unique majors", unique_majors)



                                     # depresion vs. majors 
    depression_and_majors = df.groupby(["what_is_your_course?", "do_you_have_depression?"]).size()

    yes_only = depression_and_majors.xs(
        "Yes", level="do_you_have_depression?"
    )
    top_depressed_courses = yes_only.sort_values(ascending=False).head(5)

    table = top_depressed_courses.reset_index()
    table.columns = ["course", "number of depressed individuals"]

    return table
    

    


    

data = extract()
table = transform(data)

In [48]:
# lets load our data final step

import sqlite3


In [55]:
def load(table):
    import sqlite3
    conn = sqlite3.connect("data/db/mental_health.db")
    table.to_sql(
        "depression_by_course",
        conn,
        if_exists="replace",
        index=False
    )
    conn.close()
